In [1]:
# This program use u-net to do binary segmentation on ill skin

In [1]:
%load_ext autoreload
%autoreload 1
%aimport config
from __future__ import division

import tensorflow as tf 
from glob import glob
import numpy as np
import os
from matplotlib import pyplot as plt

import config as cfg 

# from model import u_net

In [2]:
# %load model.py
from __future__ import division

import tensorflow as tf 

import config as cfg 

def Downsampling_block(inputs, filters):
    x = conv_group(inputs, filters)
    pool_1 = tf.layers.max_pooling2d(x, 2, 2)
    print pool_1.get_shape()
    return x, pool_1

def Upsampling_block(input_down, input_up, filters):
    conv_trans = tf.layers.conv2d_transpose(input_up, filters, 2, strides = 2)

#     down_feature_shape = tf.shape(input_down)
#     up_feature_shape = tf.shape(conv_trans)
#     if down_feature_shape[1] % 2 != 0 :
#         # height_pad = down_feature_shape[1].value -up_feature_shape[1].value
#         # width_pad = down_feature_shape[2].value- up_feature_shape[2].value
#         conv_trans = tf.keras.layers.ZeroPadding2D(((0, 1), (0, 0))).apply(conv_trans)
#     if down_feature_shape[2] % 2 != 0:
#         conv_trans = tf.keras.layers.ZeroPadding2D(((0, 0), (0, 1))).apply(conv_trans)

    x = tf.concat([input_down, conv_trans], axis = 3)
    x = conv_group(x, filters)
    print x.get_shape()
    return x

def conv_group(inputs, filters):
    x = tf.layers.conv2d(inputs, filters, 3,  padding = 'same')
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters, 3,  padding = 'same')
    x = tf.nn.relu(x)
    return x

def u_net(x, y_gt):

    depth = cfg.depth
    filters = cfg.filters_first
    downsample_output = []

    for i in range(depth):
        down_feature_map, x = Downsampling_block(x, filters)
        downsample_output.append(down_feature_map)
        filters = filters * 2

    x = conv_group(x, filters)
    filters = int(filters / 2)
    print x.get_shape()

    for i in range(depth):
        x = Upsampling_block(downsample_output[depth - i - 1], x, filters)
        filters = int(filters / 2)

    logits = tf.layers.conv2d(x, 2, 1)
    print logits.get_shape()

    predictions = tf.argmax(logits, axis = 3)
    print predictions.get_shape()
    
    #one-hot 
    mask = tf.constant(1, tf.int8, [cfg.batch_size, cfg.im_shape[0], cfg.im_shape[1], 1])
    bg_label = mask - y_gt
    one_hot = tf.concat([bg_label, y_gt], axis = 3)
    one_hot = tf.reshape(one_hot, [cfg.batch_size * cfg.im_shape[0] * cfg.im_shape[1], 2])
    
    logits = tf.reshape(logits, [cfg.batch_size * cfg.im_shape[0] * cfg.im_shape[1], 2])
    loss = tf.losses.softmax_cross_entropy(one_hot, logits)
    return predictions, loss

In [ ]:
def train():

    train_image, train_label, val_image, val_label = generator()
    train_image = tf.reshape(train_image, [cfg.batch_size, cfg.im_shape[0], cfg.im_shape[1], 3])
    train_label = tf.reshape(train_label, [cfg.batch_size, cfg.im_shape[0], cfg.im_shape[1], 1])
    
    predictions, loss = u_net(train_image, train_label)

    tf.summary.scalar('cross_entropy', loss)
    
    tf.logging.info("Finished loading the network.")

    global_step = tf.train.get_or_create_global_step()
    learning_rate = tf.train.exponential_decay(cfg.starter_learning_rate,
                                               global_step,
                                               cfg.decay_steps,
                                               cfg.decay_rate,
                                               staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss, global_step = global_step)
    
    acc_label = tf.reshape(train_label, [cfg.batch_size * cfg.im_shape[0] * cfg.im_shape[1], 1])
    predictions = tf.reshape(predictions, [cfg.batch_size * cfg.im_shape[0] * cfg.im_shape[1], 1])
    accuracy, acc_op = tf.metrics.accuracy(acc_label, predictions, name = 'my_metric')
    tf.summary.scalar('accuracy', accuracy)
   
    acc_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="my_metric")
    acc_vars_initializer = tf.variables_initializer(var_list=acc_vars)

    init = tf.global_variables_initializer()

    saver = tf.train.Saver()
    sess = tf.Session()
    merged = tf.summary.merge_all()
    visual_writer = tf.summary.FileWriter(cfg.visual_dir, sess.graph)

    sess.run(acc_vars_initializer)
    sess.run(init)
    
    while True:
        try:
            visual, loss_value, lr_value, acc_value, _, _ = sess.run((merged, 
                                                         loss, 
                                                         learning_rate, 
                                                         accuracy, 
                                                         acc_op,
                                                         train))
            visual_writer.add_summary(visual, tf.train.global_step(sess, global_step))
            tf.logging.info('epoch {}, iter {}, loss = {}, lr = {}, acc = {}'.format(
                            int(tf.train.global_step(sess, global_step) / cfg.train_steps),
                            tf.train.global_step(sess, global_step),
                            loss_value,
                            lr_value,
                            acc_value)
                           )
            if tf.train.global_step(sess, global_step) % cfg.model_save_step == 0 and tf.train.global_step() > 0:
                saver.save(sess, cfg.save_path, global_step)
        except tf.errors.OutOfRangeError:
            break

if __name__ == '__main__':
    train()

(8, 224, 320, 64)
(8, 112, 160, 128)
(8, 56, 80, 256)
(8, 28, 40, 512)
(8, 28, 40, 1024)
(8, 56, 80, 512)
(8, 112, 160, 256)
(8, 224, 320, 128)
(8, 448, 640, 64)
(8, 448, 640, 2)
(8, 448, 640)
INFO:tensorflow:Finished loading the network.
INFO:tensorflow:epoch 0, iter 1, loss = 0.686313152313, lr = 0.00999999977648, acc = 0.0
INFO:tensorflow:epoch 0, iter 2, loss = 0.67892152071, lr = 0.00999999977648, acc = 0.639536380768
INFO:tensorflow:epoch 0, iter 3, loss = 0.673070132732, lr = 0.00999999977648, acc = 0.739697039127
INFO:tensorflow:epoch 0, iter 4, loss = 0.663327872753, lr = 0.00999999977648, acc = 0.768449902534
INFO:tensorflow:epoch 0, iter 5, loss = 0.661952733994, lr = 0.00999999977648, acc = 0.796808958054
INFO:tensorflow:epoch 0, iter 6, loss = 0.653221428394, lr = 0.00999999977648, acc = 0.805447459221
INFO:tensorflow:epoch 0, iter 7, loss = 0.647818624973, lr = 0.00999999977648, acc = 0.814436793327
INFO:tensorflow:epoch 0, iter 8, loss = 0.633440434933, lr = 0.0099999997

INFO:tensorflow:epoch 0, iter 81, loss = 0.309118956327, lr = 0.00999999977648, acc = 0.847942650318
INFO:tensorflow:epoch 0, iter 82, loss = 0.288277179003, lr = 0.00999999977648, acc = 0.848629951477
INFO:tensorflow:epoch 0, iter 83, loss = 0.271412849426, lr = 0.00999999977648, acc = 0.848861455917
INFO:tensorflow:epoch 0, iter 84, loss = 0.491199374199, lr = 0.00999999977648, acc = 0.849507272243
INFO:tensorflow:epoch 0, iter 85, loss = 0.257118046284, lr = 0.00999999977648, acc = 0.848830759525
INFO:tensorflow:epoch 0, iter 86, loss = 0.26137149334, lr = 0.00999999977648, acc = 0.849395334721
INFO:tensorflow:epoch 0, iter 87, loss = 0.447778612375, lr = 0.00999999977648, acc = 0.850194692612
INFO:tensorflow:epoch 0, iter 88, loss = 0.448325783014, lr = 0.00999999977648, acc = 0.849945247173
INFO:tensorflow:epoch 0, iter 89, loss = 0.270705550909, lr = 0.00999999977648, acc = 0.849632143974
INFO:tensorflow:epoch 0, iter 90, loss = 0.681553244591, lr = 0.00999999977648, acc = 0.8503

INFO:tensorflow:epoch 0, iter 162, loss = 0.317677915096, lr = 0.00999999977648, acc = 0.857977032661
INFO:tensorflow:epoch 0, iter 163, loss = 0.331782162189, lr = 0.00999999977648, acc = 0.858094215393
INFO:tensorflow:epoch 0, iter 164, loss = 0.377161622047, lr = 0.00999999977648, acc = 0.858316779137
INFO:tensorflow:epoch 0, iter 165, loss = 0.359289854765, lr = 0.00999999977648, acc = 0.858306646347
INFO:tensorflow:epoch 0, iter 166, loss = 0.454437732697, lr = 0.00999999977648, acc = 0.858225762844
INFO:tensorflow:epoch 0, iter 167, loss = 0.500110507011, lr = 0.00999999977648, acc = 0.857988655567
INFO:tensorflow:epoch 0, iter 168, loss = 0.315844237804, lr = 0.00999999977648, acc = 0.85753005743
INFO:tensorflow:epoch 0, iter 169, loss = 0.369589745998, lr = 0.00999999977648, acc = 0.857764363289
INFO:tensorflow:epoch 0, iter 170, loss = 0.328474760056, lr = 0.00999999977648, acc = 0.857797086239
INFO:tensorflow:epoch 0, iter 171, loss = 0.257970482111, lr = 0.00999999977648, ac

In [3]:
a = tf.constant([[1,0,1,1,0,1,0,0,1]])
a = tf.reshape(a, [9,1])
b = tf.constant([[1,0,1,1,0,0,1,1,0]])
b = tf.reshape(b, [9,1])
accuracy, acc_op = tf.metrics.accuracy(b, a, name = 'my_metric')
acc_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="my_metric")
acc_vars_initializer = tf.variables_initializer(var_list=acc_vars)
sess = tf.Session()
sess.run(acc_vars_initializer)
print sess.run((accuracy, acc_op))


(0.5555556, 0.5555556)


In [3]:
def load_image(filename):
    image_string = tf.read_file(filename[0])
    image = tf.image.decode_image(image_string)
    image = tf.image.per_image_standardization(image)
    
    label_string = tf.read_file(filename[1])
    label = tf.image.decode_image(label_string)
    label = tf.cast(label / 255, tf.int8)
    
    return image, label

def generator():

    im_paths = sorted(glob(os.path.join(cfg.data_dir, '*.jpg')))
    gt_paths = sorted(glob(os.path.join(cfg.gt_dir, '*.png')))
    dataset = zip(im_paths, gt_paths)

    split_index = int(len(dataset) * (1 - cfg.val_fraction))
    train_files = dataset[0:split_index]
    val_files = dataset[split_index:]

    train_set = tf.data.Dataset.from_tensor_slices(train_files)
    train_set = train_set.shuffle(buffer_size = split_index)
    train_set = train_set.map(load_image)
    train_set = train_set.batch(cfg.batch_size)
    train_set = train_set.repeat(cfg.epoch)
    train_iter = train_set.make_one_shot_iterator()

    val_set = tf.data.Dataset.from_tensor_slices(val_files)
    val_set = val_set.shuffle(buffer_size = len(dataset) - split_index)
    val_set = val_set.map(load_image)
    val_set = val_set.batch(cfg.batch_size)
    val_set = val_set.repeat(cfg.epoch)
    val_iter = val_set.make_one_shot_iterator()

    train_image, train_label = train_iter.get_next()
    val_image, val_label =val_iter.get_next()

    return train_image, train_label, val_image, val_label

# Testing
# sess = tf.Session()
# batch = sess.run(train_iter.get_next())
# im = batch[0][0]
# gt = batch[1][0] * 255
# print im.shape, gt.shape
# s = np.zeros((gt.shape), dtype = int)
# gt = np.concatenate((gt, s, s), axis = 2)

# plt.imsave('im.jpg',im)
# plt.figure()
# plt.imsave('gt.png', gt)

In [4]:
from __future__ import division
print 16*56*160*1024 / 1000000000

0.14680064


In [ ]:
train_writer = tf.python_io.TFRecordWriter(cfg.train_record_path)
val_writer = tf.python_io.TFRecordWriter(cfg.val_record_path)

im_paths = sorted(glob(os.path.join(cfg.data_dir, '*.jpg')))
gt_paths = sorted(glob(os.path.join(cfg.gt_dir, '*.png')))

split_index = int(len(im_paths) * (1 - cfg.val_fraction))
print split_index

# for i in range(len(im_paths)):
for i in range(1):
    im = Image.open(im_paths[i])
    im_data = np.resize(np.array(im) / 255, cfg.im_shape)
    plt.figure()
    plt.imshow(im)
    im_data = im_data.tostring()
    
    test = tf.decode_raw(im_data, tf.float32)
    
    gt = Image.open(gt_paths[i])
    gt_data = np.resize(np.array(gt) / 255, cfg.im_shape[0:-1])
    gt_data = np.array(gt_data, dtype = int)
    gt_data = gt_data.tostring()
    
    images = tf.train.Feature(bytes_list = tf.train.BytesList(value = [im_data]))
    labels = tf.train.Feature(bytes_list = tf.train.BytesList(value = [gt_data]))
    im_shape = tf.train.Feature(int64_list = tf.train.Int64List(value = [cfg.im_shape[0],cfg.im_shape[1],cfg.im_shape[2]]))
    
    features = tf.train.Features(feature = {
        "images" : images,
        "labels" : labels,
        "im_shape" : im_shape
    })
    
    imageFeatures = tf.train.Example(features = features)
    record = imageFeatures.SerializeToString()
    
    if i <= split_index:
        train_writer.write(record)
    else:
        val_writer.write(record)
    
    

In [ ]:
im_paths = sorted(glob(os.path.join(cfg.gt_dir, '*.png')))
im = Image.open(im_paths[0])
plt.imshow(im)
im_data = np.array(im)
print im_data.shape
new = np.array(im_data / 255, dtype = int)

new = new * 255
# plt.figure()
# plt.imshow(new_im)
# # with open('see.txt', 'a') as f:
# #     for i in range(new.shape[0]) :
# #         for j in range(new.shape[1]):
# #             f.write(str(new[i][j]) + ' ')

test = im_data - new
print test.shape
print sum(sum(test))
# #             if im_data[i][j] == 255:
# #                 im_data[i][j] = 0
# # print sum(sum(im_data))
new_im = Image.fromarray(test, mode = 'L')
new_im.save('1.png')
fanren = Image.open('1.png')
fanren = np.array(fanren)
s = fanren
print s.shape
print sum(sum(s))

new = new.reshape((928,1344,1))
s = np.zeros(new.shape, dtype = int)
test = np.concatenate((new, s, s), axis=2)

plt.imsave('qiguai.png', test)


In [5]:
def parseExample(example_ptoto):
    feature_dict = {
        "images" : tf.FixedLenFeature(shape = [], dtype = tf.string), 
        "labels" : tf.FixedLenFeature(shape = [], dtype = tf.string),
        "im_shape" : tf.FixedLenFeature(shape = (3), dtype = tf.int64)
    }
    features = tf.parse_single_example(example_ptoto, feature_dict)
    
#     features = tf.train.example(features)
#     features.ParseFromString(featureString)

    im_shape = features['im_shape']
    print im_shape
    
#     images = tf.string_to_number(features['images'], tf.float32)
    images = tf.decode_raw(features['images'], tf.float32)
    images = tf.reshape(images, im_shape)
#     labels = tf.string_to_number(features['labels'], tf.int32)
    
    labels = tf.decode_raw(features['labels'], tf.int32)
    labels = tf.reshape(labels, im_shape[0:-1])
    
    return images, labels
    
dataset = tf.data.TFRecordDataset(cfg.train_record_path)
dataset = dataset.map(parseExample)
# dataset = dataset.batch(1)

iterator = dataset.make_one_shot_iterator()
# images, labels = iterator.get_next()

# sess = tf.Session()
# sess.run(images)
# print images.shape()

# dataset = dataset.shuffle()
# dataset = dataset.batch(cfg.batch_size)
# dataset = dataset.repeat(cfg.epoch)

# prediction, loss = u_net(dataset)

# optimizer = tf.train.GradientDescentOptimizer(cfg.learning_rate)
# train = optimizer.minimize(loss)
# accuracy, _ = tf.metrics.accuracy(y_gt[:,:,:,1], predictions, name = 'my_metric')


Tensor("ParseSingleExample/Squeeze_im_shape:0", shape=(3,), dtype=int64)
